In [69]:
import boto3
import json
from typing import Dict, Any
from dotenv import load_dotenv
import os

In [70]:
load_dotenv()
region = os.environ.get('AWS_REGION')
MODEL_ID = os.environ.get('MODEL_ID')
role_arn = os.environ.get('ROLE_ARN')
knowledge_base_id = os.environ.get('KNOWLEDGE_BASE_ID')

In [71]:
bedrock_agent = boto3.client('bedrock-agent', region_name = region)
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name = region)

In [72]:
flow_name="Routers"
flow_description="A example of routing based on condition"

In [73]:
# The input node validates that the content of the InvokeFlow request is a JSON object.
input_node = {
    "type": "Input",
    "name": "FlowInput",
    "outputs": [
        {
            "name": "document",
            "type": "String"
        }
    ]
}

In [74]:
intent_prompt = "Identify the intent from the text provided.  If the question is related to Agentic Memory, say 'agents' if not say 'others'  just provide the intent.  do not add any preambles.  Question: {{query}}"

intent_node = {
    "type": "Prompt",
    "name": "detectintent",
    "configuration": {
        "prompt": {
            "sourceConfiguration": {
                "inline": {
                    "modelId": MODEL_ID,
                    "templateType": "TEXT",
                    "inferenceConfiguration": {
                        "text": {
                            "temperature": 0.8
                        }
                    },
                    "templateConfiguration": { 
                        "text": {
                            "text": intent_prompt
                        }
                    }
                }
            }
        }
    },
    "inputs": [
        {
            "name": "query",
            "type": "String",
            "expression": "$.data"
        },
    ],
    "outputs": [
        {
            "name": "modelCompletion",
            "type": "String"
        }
    ]
}

In [75]:
conditional_node = {
        "name": "ConditionalNode",
        "type": "Condition",
        "configuration": {
            "condition": {
                "conditions": [
                    {
                        "expression": "conditionInput == \"agents\"",
                        "name": "intent"
                    },
                    {
                        "name": "default"
                    }
                ]
            }
        },
        "inputs": [
            {
                "name": "conditionInput",
                "type": "String",
                "expression": "$.data"

            }
        ],

    }

In [76]:
web_search_node = {
        "name": "web_search",
        "type": "LambdaFunction",
        "configuration" : {
            'lambdaFunction': {
                'lambdaArn': 'arn:aws:lambda:us-west-2:924155096146:function:web_search'
            },
        },
        "inputs": [
            {
                "name": "codeHookInput",
                "type": "String",
                "expression": "$.data"
            }
        ],
        "outputs": [
            {
                "name": "functionResponse",
                "type": "String"
            }
        ]
}

In [77]:
kb_search_node = {
        "name": "kb_search_node",
        "type": "KnowledgeBase",
        "configuration" : {
            'knowledgeBase': {
                'knowledgeBaseId': knowledge_base_id,
                'modelId': MODEL_ID
            }
        },
        "inputs": [
            {
                "name": "retrievalQuery",
                "type": "String",
                "expression": "$.data"
            }
        ],
        "outputs": [
            {
                "name": "outputText",
                "type": "String"
            }
        ]
        
}

In [78]:
blogger_node = {
    "type": "Prompt",
    "name": "blogger",
    "configuration": {
        "prompt": {
            "sourceConfiguration": {
                "inline": {
                    "modelId": MODEL_ID,
                    "templateType": "TEXT",
                    "inferenceConfiguration": {
                        "text": {
                            "temperature": 0.8
                        }
                    },
                    "templateConfiguration": { 
                        "text": {
                            "text": "Create a succint summary from the context: {{webresults}}."
                        }
                    }
                }
            }
        }
    },
    "inputs": [
        {
            "name": "webresults",
            "type": "String",
            "expression": "$.data"
        }
    ],
    "outputs": [
        {
            "name": "modelCompletion",
            "type": "String"
        }
    ]
}

In [79]:
output_node = {
    "type": "Output",
    "name": "FlowOutput",
    "inputs": [
        {
            "name": "document",
            "type": "String",
            "expression": "$.data"
        }
    ]
}

In [80]:
output_node2 = {
    "type": "Output",
    "name": "FlowOutput2",
    "inputs": [
        {
            "name": "document",
            "type": "String",
            "expression": "$.data"
        }
    ]
}

In [83]:
connections =  [
    {
      "name": "Input_to_intent",
      "source": input_node['name'],
      "target": intent_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": input_node['outputs'][0]['name'], 
            "targetInput": intent_node['inputs'][0]['name']
          }
      }
    },
    {
      "name": "intent_to_condition",
      "source": intent_node["name"],
      "target": conditional_node["name"],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": intent_node['outputs'][0]['name'], 
            "targetInput": conditional_node['inputs'][0]['name']
          }
      }
    },
    {
      "name": "condition_to_kbnode",
      "source": conditional_node["name"],
      "target": kb_search_node["name"],
      "type": "Conditional",
      "configuration": {
        "conditional": { 
            "condition": "intent"
          }
      }
    },
    {
      "name": "condition_to_websearch",
      "source": conditional_node["name"],
      "target": web_search_node["name"],
      "type": "Conditional",
      "configuration": {
        "conditional": { 
            "condition": "default"
          }
      }
    },
    {
      "name": "Input_to_kbnode",
      "source": input_node['name'],
      "target": kb_search_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": input_node['outputs'][0]['name'], 
            "targetInput": kb_search_node['inputs'][0]['name']
          }
      }
    },
    {
      "name": "Input_to_websearch",
      "source": input_node['name'],
      "target": web_search_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": input_node['outputs'][0]['name'], 
            "targetInput": web_search_node['inputs'][0]['name']
          }
      }
    },
    {
      "name": "websearch_to_blog",
      "source": web_search_node['name'],
      "target": blogger_node['name'],
      "type": "Data",
      "configuration": {
        "data": { 
            "sourceOutput": web_search_node['outputs'][0]['name'], 
            "targetInput": blogger_node['inputs'][0]['name'],
          },
      }
    },
    {
      "name": "blog_to_output",
      "source": blogger_node['name'],
      "target": output_node['name'],
      "type": "Data",
      "configuration": {
        "data": {
          "sourceOutput": blogger_node['outputs'][0]['name'],
          "targetInput": output_node['inputs'][0]['name']
        }
      }
    },
    {
      "name": "kb_to_output",
      "source": kb_search_node['name'],
      "target": output_node2['name'],
      "type": "Data",
      "configuration": {
        "data": {
          "sourceOutput": kb_search_node['outputs'][0]['name'],
          "targetInput": output_node2['inputs'][0]['name']
        }
      }
    }
]

In [84]:
response = bedrock_agent.create_flow(
    name=flow_name,
    description=flow_description,
    executionRoleArn=role_arn,
    definition={
        "nodes": [input_node, intent_node, conditional_node, web_search_node, kb_search_node, blogger_node, output_node, output_node2],
        "connections": connections
    }
)

In [66]:
flow_id = response.get("id")

In [ ]:
bedrock_agent.prepare_flow(flowIdentifier=flow_id)

In [ ]:
response = bedrock_agent.create_flow_version(flowIdentifier=flow_id)
                                
flow_version = response.get("version")

In [ ]:
response = bedrock_agent.create_flow_alias(
    flowIdentifier=flow_id,
    name="latest",
    description="Alias pointing to the latest version of the flow.",
    routingConfiguration=[
        {
            "flowVersion": flow_version
        }
    ]
)

flow_alias_id = response.get("id")

In [ ]:
response = bedrock_agent_runtime.invoke_flow(
    flowIdentifier=flow_id,
    flowAliasIdentifier=flow_alias_id,
    inputs=[
        {
            "content": {
                "document": "agentic memory"
            },
            "nodeName": "FlowInput",
            "nodeOutputName": "document"
        }
    ]
)

result = {}

for event in response.get("responseStream"):
    result.update(event)

if result['flowCompletionEvent']['completionReason'] == 'SUCCESS':
    print("Flow invocation was successful! The output of the flow is as follows:\n")
    print(result['flowOutputEvent']['content']['document'])

else:
    print("The flow invocation completed because of the following reason:", result['flowCompletionEvent']['completionReason'])